In [1]:
import numpy as np
import matplotlib.pyplot as plt
from bnt_smooth import LognormalWeakLensingSim

# --- Define a parent n(z) and tomographic binning ---
def parent_nz(z):
    return z**2 * np.exp(-(z / 0.5)**1.5)

def make_equal_ngal_bins(nz_func, z, nbins):
    from scipy.interpolate import interp1d
    nz = nz_func(z)
    nz /= np.trapz(nz, z)
    cdf = np.cumsum(nz)
    cdf /= cdf[-1]
    inv_cdf = interp1d(np.concatenate([[0], cdf, [1]]),
                       np.concatenate([[z[0]], z, [z[-1]]]))
    edges = inv_cdf(np.linspace(0, 1, nbins + 1))
    bins = []
    for i in range(nbins):
        mask = (z >= edges[i]) & (z <= edges[i+1])
        bins.append((z[mask], nz[mask]))
    return bins

# --- Set simulation parameters ---
z = np.linspace(0.01, 2.5, 500)
nz_list = make_equal_ngal_bins(parent_nz, z, nbins=3)
n_eff_list = [30.0, 30.0, 30.0]       # galaxies per arcmin²
sigma_eps_list = [0.26, 0.26, 0.26]
lognormal_shifts = [0.0, 0.0, 0.0]
baryon_feedback = 3.13
sigma8 = 0.8
seed = 1234
l_max = 3000

# --- Initialize simulation ---
sim = LognormalWeakLensingSim(
    nz_list=nz_list,
    n_eff_list=n_eff_list,
    sigma_eps_list=sigma_eps_list,
    baryon_feedback=baryon_feedback,
    sigma8=sigma8,
    lognormal_shifts=lognormal_shifts,
    seed=seed,
    l_max=l_max
)

# --- Generate fields ---
maps = sim.generate_lognormal_fields(n_pix=256, map_size_deg=5.0)

RuntimeError: This version of jaxlib was built using AVX instructions, which your CPU and/or operating system do not support. This error is frequently encountered on macOS when running an x86 Python installation on ARM hardware. In this case, try installing an ARM build of Python. Otherwise, you may be able work around this issue by building jaxlib from source.